In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def logic_win(x,x_r):
    return (x >= x_r[0]) & (x < x_r[1])

#### Import and clean dataset, select out London

In [3]:
path = '../Data/UK/combined_collisions.csv'

df = pd.read_csv(path)
df.drop(columns='Unnamed: 0',inplace=True)

In [4]:
#Time column has some nans in it, must drop those entries

def is_str(x):
    Nx = len(x)
    I = np.ones(Nx).astype(bool)
    for ii in range(Nx):
        if type(x[ii]) != str:
            I[ii] = False
    return I

df = df.loc[is_str(df.Time.values),:]
df.aadf_Count_point_id = df.aadf_Count_point_id.values.astype(int)

In [5]:
#make hour, year and month columns
    #hour will be rounded 
def get_hour(T):
    Nt = len(T)
    H = np.zeros(Nt).astype(int)
    for ii in range(Nt):
        s = T[ii].split(':')
        h = int(s[0])
        m = int(s[1])
        if m > 30:
            h += 1
        H[ii] = h % 24
    return H

def get_ymd(D):
    Nd = len(D)
    # |
    ymd = np.zeros((Nd,3)).astype(int)
    for ii in range(Nd):
        s = D[ii].split('-')
        for nn in range(3):
            ymd[ii,nn] = int(s[nn])
    return ymd

def reformat_aadf_year(Year):
    Ny = len(Year)
    y = np.zeros(Ny).astype(int)
    for ii in range(Ny):
        y[ii] = int(Year[ii].split('-')[0])
    return y

df[['y','m','d']] = get_ymd(df.Date.values)
df['h'] = get_hour(df.Time.values)
#df['aadf_Year'] = reformat_aadf_year(df.aadf_Year.values)

In [6]:
#central London box
Lat_r = [51.475,51.543]
Lon_r = [-0.179,0.023]

I_lat = (df.Latitude > Lat_r[0]) & (df.Latitude < Lat_r[1])
I_lon = (df.Longitude > Lon_r[0]) & (df.Longitude < Lon_r[1])
I = I_lat & I_lon

df_lond = df.loc[I,:].copy()

In [7]:
#drop all data that don't have traffic matches

I = df_lond.match == True
df_lond = df_lond.loc[I,:]
df_lond.aadf_Year = reformat_aadf_year(df_lond.aadf_Year.values)

#### Make neighborhoods feature

In [8]:
#Make a 3 (Lat, N/S) x 6 (Lon, E/W) grid
Nlat = 3
Nlon = 6
Nneigh = Nlon*Nlat

edges_lat = np.linspace(*Lat_r,Nlat+1)
dlat = np.diff(edges_lat)[0]
bins_lat = edges_lat[:-1]+0.5*dlat
edges_lon = np.linspace(*Lon_r,Nlon+1)
dlon = np.diff(edges_lon)[0]
bins_lon = edges_lon[:-1]+0.5*dlon

coord_neigh = [None]*Nneigh

df_lond['neigh'] = 0 #initialize neighborhood feature
ir = 0 #these are lat bins (rows)
ic = 0 #lon bins
for ii in range(Nneigh):
    Ilat = logic_win(df_lond.Latitude,(edges_lat[ir],edges_lat[ir+1]))
    Ilon = logic_win(df_lond.Longitude,(edges_lon[ic],edges_lon[ic+1]))
    I = Ilat & Ilon
    df_lond.loc[I,'neigh'] = ii
    
    coord_neigh[ii] = np.array([bins_lat[ir],bins_lon[ic]])
    
    ic+=1
    if ic == Nlon:
        ic = 0
        ir+=1

#### Make a DataFrame using df_lond that has entries only for traffic IDs. Main feature is # of incidents.¶
- Other features are the aadf features that go along with the ID
- Also maybe some averaged features from accidents dataset (like junction control, maybe just use the classification with the most entries for that ID)
- Maybe the target variable should be number of incidents normalized by the bike volume (this is prob. of an incident)

##### Features to include
- all aadf features (do they neeed to be averaged over year? Maybe just average anyway)
- modes of classification features in accidents
- num. of incidents
- num. of serious and fatal incidents
- num. of incidents normalized to bike volume from aadf

In [19]:
traf_IDs = np.array(df_lond.aadf_Count_point_id.value_counts().index)
Ninc_IDs = np.array(df_lond.aadf_Count_point_id.value_counts().values)

Nthresh = 1 #this sets which traffic counters are included, they must have at least this many accidents 

I = Ninc_IDs >= Nthresh #this will select roads to include in the new DataFrame based on if they had at least this many accidents
I.sum()

IDs_df = traf_IDs[I]
Nid = len(IDs_df)

df_road = pd.DataFrame({'ID':IDs_df})
slight_rd = np.zeros(Nid).astype(int)
ser_rd = np.zeros(Nid).astype(int)
total_rd = np.zeros(Nid).astype(int)
for nn,ii in enumerate(IDs_df):
    I = df_lond.aadf_Count_point_id.values == ii
    total_rd[nn] = I.sum()
    slight_rd[nn] = df_lond.loc[I,'slight'].values.sum()
    ser_rd[nn] = total_rd[nn] - slight_rd[nn]
df_road['total'] = total_rd
df_road['slight'] = slight_rd
df_road['serious'] = ser_rd

safety_mode = ['neigh','Road_name','Road_name2','h','Day_of_Week','m','Speed_limit','Junction_Detail','Junction_Control','Pedestrian_Crossing-Physical_Facilities']
aadf_av = ['distance_to_cp','aadf_Pedal_cycles','aadf_Pedal_cycles','aadf_All_motor_vehicles','aadf_All_HGVs','aadf_LGVs','aadf_Buses_and_coaches']
aadf_mode = ['aadf_Road_name','aadf_Road_category','aadf_Road_type']
for f in safety_mode:
    df_road[f] = np.zeros(Nid).astype(int)
    for nn,ii in enumerate(IDs_df):
        I = df_lond.aadf_Count_point_id.values == ii
        df_road.loc[nn,f] = df_lond.loc[I,f].mode().values[0]

for f in aadf_mode:
    df_road[f] = np.zeros(Nid).astype(int)
    for nn,ii in enumerate(IDs_df):
        I = df_lond.aadf_Count_point_id.values == ii
        df_road.loc[nn,f] = df_lond.loc[I,f].mode().values[0]
        
for f in aadf_av:
    df_road[f] = np.zeros(Nid).astype(int)
    for nn,ii in enumerate(IDs_df):
        I = df_lond.aadf_Count_point_id.values == ii
        df_road.loc[nn,f] = df_lond.loc[I,f].mean()
        
df_road['total_norm'] = df_road['total'].values/df_road['aadf_Pedal_cycles'].values
df_road['serious_ratio'] = df_road['serious'].values/df_road['total'].values

<ipython-input-19-56545ccd8477>:46: RuntimeWarning: divide by zero encountered in divide
  df_road['total_norm'] = df_road['total'].values/df_road['aadf_Pedal_cycles'].values


In [20]:
#add in aadf year availability metadata 
path_year = '../Data/UK/aadf_year_meta.csv'
df_year = pd.read_csv(path_year)

#merge the metadata into df_road
feat_add = list(df_year.columns)
feat_add.remove('Count_point_id')
for c in feat_add:
    df_road[c] = np.zeros(Nid).astype(int)
    for ii,ID in enumerate(df_road.ID.values):
        I = df_year.Count_point_id == ID
        df_road.loc[ii,c] = df_year.loc[I,c].values

In [21]:
df_road.sample(20)

,ID,total,slight,serious,neigh,Road_name,Road_name2,h,Day_of_Week,m,...,aadf_All_motor_vehicles,aadf_All_HGVs,aadf_LGVs,aadf_Buses_and_coaches,total_norm,serious_ratio,N_year,Year_start,Year_end,Max_year_gap
213,18600,21,19,2,8,A4,A40,9,3,9,...,13347.333333,473.857143,1847.285714,254.904762,0.015707,0.095238,22,2000,2021,1
420,74977,5,5,0,12,A41,A501,9,3,10,...,16751.400000,399.600000,2058.800000,1639.200000,0.017743,0.000000,22,2000,2021,1
37,46432,72,63,9,8,A40,C,18,5,5,...,25925.375000,619.472222,2549.277778,1644.680556,0.017061,0.125000,22,2000,2021,1
306,26146,12,11,1,6,A5,U,8,2,10,...,34576.166667,1080.333333,5825.250000,2244.250000,0.008435,0.083333,22,2000,2021,1
316,56433,12,11,1,0,A3214,na0,9,5,3,...,25325.583333,757.833333,2553.583333,2966.333333,0.010740,0.083333,22,2000,2021,1
291,18736,14,13,1,11,A1206,U,17,3,10,...,33132.428571,1599.642857,4988.000000,417.428571,0.139007,0.071429,22,2000,2021,1
50,47786,62,52,10,14,A5201,U,9,4,3,...,21997.500000,933.387097,3967.209677,759.387097,0.012345,0.161290,22,2000,2021,1
83,48687,49,42,7,8,A10,A1211,18,4,4,...,19774.367347,822.469388,3108.469388,1722.734694,0.011728,0.142857,22,2000,2021,1
442,26464,3,1,2,12,A41,A501,8,3,3,...,14369.666667,401.333333,1342.333333,1385.666667,0.005172,0.666667,22,2000,2021,1
134,37774,35,28,7,7,A4201,A4201,18,5,8,...,13921.571429,583.200000,2743.514286,1176.857143,0.036403,0.200000,22,2000,2021,1


In [22]:
save_path = '../DataFrames/df_road.csv'
df_road.to_csv(save_path,index_label=False)

In [18]:
df_lond.distance_to_cp.describe()

count    1.373500e+04
mean     3.144804e-05
std      3.175884e-05
min      2.279401e-08
25%      9.723167e-06
50%      2.048280e-05
75%      4.267008e-05
max      2.800514e-04
Name: distance_to_cp, dtype: float64